In [0]:

import numpy as np
import sklearn.metrics as metrics
import os, os.path
import pandas as pd
import math
np.random.seed(2019)
import time


from keras.datasets import cifar10
import keras.callbacks as callbacks
from keras.utils import np_utils
from keras.layers import Input
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

from keras import backend as K
from keras.backend.tensorflow_backend import set_session
#import tensorflow as tf

In [0]:
import keras.models
from keras import regularizers
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout,Conv2D
from keras.layers.normalization import BatchNormalization


## **Load Dataset**

In [3]:
(trainX, trainY), (testX, testY) = cifar10.load_data()
num_classes = len(np.unique(trainY))

170500096/170498071 [==============================] - 11s 0us/step


### used for slicing the image(tensorflow function)

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainY))
test_dataset = tf.data.Dataset.from_tensor_slices((testX, testY))

**Get shape of training and test images**

In [5]:
len_train, len_test = len(trainX), len(testX)
print(trainX.shape, trainY.shape)

(50000, 32, 32, 3) (50000, 1)


In [0]:
#scale the test and train data

trainX = trainX.astype('float32')/255
testX = testX.astype('float32')/255

In [0]:
batch_size = 64
nb_epoch = 50
img_rows, img_cols = 32, 32

**standard scaling: subtract by mean, and divide by standard deviation**

In [8]:
# mean value 
train_mean=np.array([0.4914, 0.4822, 0.4465])
train_mean

array([0.4914, 0.4822, 0.4465])

In [9]:
# std value 
train_std=np.array([0.2023, 0.1994, 0.2010])
train_std

array([0.2023, 0.1994, 0.201 ])

In [0]:
# function to normalize train and test image
trainX = (trainX - train_mean) / train_std
testX = (testX - train_mean) / train_std

**In DavidNet**


training images go through the standard Cifar10 transformations, that is: 

* pad 4 pixels to 40×40, crop back to 32×32.

* apply random crop to the image

* randomly flip left and right. 

* In addition, it apply the popular Cutout augmentation as a regularization measure, which alleviates overfitting. 

### function to pad training image by 4 pixel

In [0]:
def pad4(x):
    return np.pad(x, ((0,0), (4, 4), (4, 4),(0,0)), mode='reflect')

In [0]:
trainX = pad4(trainX)


###Function to randomly crop the training image and random flips of the image(tensorflow function)

In [0]:
#Function to randomly crop the training image and random horizontal flips of the image
#data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
## map function to apply the augmentation to each element
#trainset = train_dataset.map(data_aug).shuffle(len_train).batch(batch_size).prefetch(1)

In [0]:
#train_set

### **keras random crop function **

In [0]:
def random_crop(x, random_crop_size = (32,32), sync_seed=None):
    np.random.seed(sync_seed)
    w, h = x.shape[1], x.shape[2]
    rangew = (w - random_crop_size[0]) // 2
    rangeh = (h - random_crop_size[1]) // 2
    offsetw = 0 if rangew == 0 else np.random.randint(rangew)
    offseth = 0 if rangeh == 0 else np.random.randint(rangeh)
    return x[:, offsetw:offsetw+random_crop_size[0], offseth:offseth+random_crop_size[1]]

In [0]:
train_X=random_crop(trainX)

### convert class labels to binary class labels

In [0]:
trainY = np_utils.to_categorical(trainY, num_classes)
testY = np_utils.to_categorical(testY, num_classes)

### Build Keras Model

In [0]:
# Define the model
def model1():
  
  model = Sequential()
  model.add(Conv2D(32, (3, 3), border_mode='same',kernel_regularizer=regularizers.l2(0.0001),name='conv2D_1', input_shape=(32, 32, 3)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(64, (3, 3),kernel_regularizer=regularizers.l2(0.0001),name='conv2D_2',border_mode='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(32, (1, 1),name='conv2D_3'))


  model.add(Conv2D(64, (3, 3),kernel_regularizer=regularizers.l2(0.0001),name='conv2D_4',border_mode='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(128, (3, 3),kernel_regularizer=regularizers.l2(0.0001),name='conv2D_5',border_mode='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.3))

  model.add(Conv2D(32, (1, 1),name='conv2D_6'))
  
  model.add(Conv2D(128, (3, 3),kernel_regularizer=regularizers.l2(0.0001),name='conv2D_7', border_mode="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(256, (3, 3),kernel_regularizer=regularizers.l2(0.0001),name='conv2D_8', border_mode="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))

  model.add(Conv2D(10, (1, 1),name='conv2D_9'))

  model.add(AveragePooling2D(pool_size = (4,4)))
  model.add(Flatten())


  model.add(Activation('softmax'))
  return model


In [23]:
model = model1()

W0810 14:25:39.009887 140378280564608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_regularizer=<keras.reg..., name="conv2D_1", input_shape=(32, 32, 3..., padding="same")`
  after removing the cwd from sys.path.
W0810 14:25:39.016397 140378280564608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 14:25:39.019019 140378280564608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 

In [24]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0810 14:26:39.019538 140378280564608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [26]:
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras import optimizers

model = model1()

filepath='saved_model1'

checkpointer = ModelCheckpoint(filepath=filepath, monitor='val_acc', mode='auto', verbose = 1, save_best_only=True)
lr_reducer = ReduceLROnPlateau(monitor='val_acc',factor=0.8, cooldown=0, patience=5, min_lr=0.5e-9,verbose = 1)

# Compile the model
sgd = optimizers.SGD(lr=0.01, decay=0, momentum=0.9, nesterov=False)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_regularizer=<keras.reg..., name="conv2D_1", input_shape=(32, 32, 3..., padding="same")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), kernel_regularizer=<keras.reg..., name="conv2D_2", padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), kernel_regularizer=<keras.reg..., name="conv2D_4", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), kernel_regularizer=<keras.reg..., name="conv2D_5", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Updat

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(zoom_range=0.0, 
                             horizontal_flip=False)
                             

# train the model
start = time.time()
# Train the model
model_info = model.fit_generator(datagen.flow(train_X, trainY, batch_size = 128),
                                 samples_per_epoch = trainX.shape[0], nb_epoch = 100, 
                                 validation_data = (testX, testY),callbacks=[checkpointer,lr_reducer], verbose=1)
end = time.time()
print ("Model took %0.2f seconds to train"%(end - start))
# plot model history
plot_model_history(model_info)
# compute test accuracy
print ("Accuracy on test data is: %0.2f"%accuracy(X_test, Y_test, model))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., verbose=1, steps_per_epoch=390, epochs=100)`
  if sys.path[0] == '':
W0810 14:29:48.360789 140378280564608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_o

Epoch 1/100
390/390 [==============================] - 32s 83ms/step - loss: 1.5337 - acc: 0.4634 - val_loss: 1.3650 - val_acc: 0.5235

Epoch 00001: val_acc improved from -inf to 0.52350, saving model to saved_model1
Epoch 2/100
390/390 [==============================] - 28s 73ms/step - loss: 1.1304 - acc: 0.6111 - val_loss: 1.1095 - val_acc: 0.6183

Epoch 00002: val_acc improved from 0.52350 to 0.61830, saving model to saved_model1
Epoch 3/100
390/390 [==============================] - 28s 72ms/step - loss: 0.9827 - acc: 0.6683 - val_loss: 0.9979 - val_acc: 0.6630

Epoch 00003: val_acc improved from 0.61830 to 0.66300, saving model to saved_model1
Epoch 4/100
390/390 [==============================] - 29s 73ms/step - loss: 0.8745 - acc: 0.7066 - val_loss: 0.9144 - val_acc: 0.7012

Epoch 00004: val_acc improved from 0.66300 to 0.70120, saving model to saved_model1
Epoch 5/100
390/390 [==============================] - 29s 74ms/step - loss: 0.7862 - acc: 0.7382 - val_loss: 0.9250 - val_